<a href="https://colab.research.google.com/github/bobby360/ML_Stage4/blob/master/Stage4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up kaggle.

In [0]:
#Setting up Kaggle
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"bobbyml","key":"ac9a2a8daab26dce1c864312a6705acf"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
#Downloading the dataset
!kaggle datasets download -d snap/amazon-fine-food-reviews -p /content

amazon-fine-food-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [74]:
#Unzipping
!unzip /content/amazon-fine-food-reviews.zip

Archive:  /content/amazon-fine-food-reviews.zip
replace Reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: Reviews.csv             Y

replace database.sqlite? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: database.sqlite         Y

replace hashes.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: hashes.txt              


In [0]:
import pandas as pd
import numpy
import sqlite3

### **TASK 1 and 2: filtering reviews; Produce a schema where count is greater than one.**



In [76]:
# Positive review Score > 3, Negative review Score < 3
CONN = sqlite3.connect("database.sqlite")

DF = pd.read_sql_query("""select UserId, ProductId, 
                          ProfileName, Time,
                          Score, Text, COUNT(*)
                          FROM reviews
                          WHERE Score != 3
                          GROUP BY UserId
                          HAVING COUNT(*) > 1""", CONN)
DF.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B007Y59HVM,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ET0,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B007Y59HVM,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ET0,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBE1U,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


### **TASK 3: Sort product id in ascending order and deduplicate data.**

In [77]:
DF.sort_values('ProductId', inplace=True)
DF.drop_duplicates(subset="ProductId",
                   keep=False, inplace=True)
DF.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,"The same author wrote ""Where the Wild Things A...",2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,"It's just as described. Its minty, helps with ...",2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,This product is Awesome and Helpful for anyone...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,Naturade Total Soy Meal Replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,Buying this much is the most cost effective wa...,2


### **TASK 4: How many positive and negative reviews are present in our dataset**

In [78]:
# Positive review scores: 4,5; Negative review scores 1,2
POS_REV = DF['Score'].value_counts()[4] + DF['Score'].value_counts()[5]
NEG_REV = DF['Score'].value_counts()[1] + DF['Score'].value_counts()[2]

print("There are " + str(POS_REV), 'positive reviews in the dataset')
print("There are " + str(NEG_REV), 'negative reviews in the dataset')

There are 9949 positive reviews in the dataset
There are 1844 negative reviews in the dataset


### **TASK 5: Remove the html tags**



In [79]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    S = MLStripper()
    S.feed(html)
    return S.get_data()

DF['Text'] = DF['Text'].apply(strip_tags)

DF.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,"The same author wrote ""Where the Wild Things A...",2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,"It's just as described. Its minty, helps with ...",2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,This product is Awesome and Helpful for anyone...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,Naturade Total Soy Meal Replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,Buying this much is the most cost effective wa...,2


### **TASK 6: Remove any punctuations or limited set of special characters**

In [80]:
import re

def remove_symbols(text):
    return re.sub("[^A-z0-9\s]+", "", text)

DF['Text'] = DF['Text'].apply(remove_symbols)

DF.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,The same author wrote Where the Wild Things Ar...,2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,Its just as described Its minty helps with my ...,2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,This product is Awesome and Helpful for anyone...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,Naturade Total Soy Meal Replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,Buying this much is the most cost effective wa...,2


### **TASK 7: Check if the word is made up of English letters and is not alpha-numeric**

In [82]:
def letters_only(text):
    return re.sub("[^a-zA-Z\s]", "", text)

DF['Text'] = DF['Text'].apply(letters_only)

DF.head()


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,The same author wrote Where the Wild Things Ar...,2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,Its just as described Its minty helps with my ...,2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,This product is Awesome and Helpful for anyone...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,Naturade Total Soy Meal Replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,Buying this much is the most cost effective wa...,2


### **TASK 8: Check to see if the length of the word is greater than 2**

In [83]:
def check_length(text):
    text = text.split()
    contain = []

    for words in text:
        if len(words) > 2:
            contain.append(words)

    final = ' '.join(contain)
    return final

DF['Text'] = DF['Text'].apply(check_length)

DF.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,The same author wrote Where the Wild Things Ar...,2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,Its just described Its minty helps with milddr...,2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,This product Awesome and Helpful for anyone wi...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,Naturade Total Soy Meal Replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,Buying this much the most cost effective way a...,2


### **TASK 9: Convert the word to lowercase**

In [84]:
def lower_case(text):
    return text.lower()

DF['Text'] = DF['Text'].apply(lower_case)

DF.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,the same author wrote where the wild things ar...,2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,its just described its minty helps with milddr...,2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,this product awesome and helpful for anyone wi...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,naturade total soy meal replacement powder was...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,buying this much the most cost effective way a...,2


### **TASK 10: Remove Stopwords**

In [85]:
import nltk

def remove_stopwords(text):
    try:
        from nltk.corpus import stopwords
        from nltk.tokenize import word_tokenize
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(text)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]

    except:
        print("Downloading NLTK resources")
        nltk.download("stopwords")
        nltk.download("punkt")
        return remove_stopwords(text)

    return " ".join(filtered_sentence)

DF['Text'] = DF['Text'].apply(remove_stopwords)


DF.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,author wrote wild things carol king wrote grea...,2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,described minty helps milddry mouth provides r...,2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,product awesome helpful anyone asthma breathin...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,naturade total soy meal replacement powder adv...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,buying much cost effective way since lot makes...,2


### **TASK 11: Finally Snowball Stemming the word**

In [87]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
def snow_stem(text):

    STEMMER = SnowballStemmer('english')
    WORD_TOKENS = nltk.word_tokenize(text)
    STEMMED_WORD = [STEMMER.stem(word) for word in WORD_TOKENS]
    return " ".join(STEMMED_WORD)

DF['Text'] = DF['Text'].apply(snow_stem)

DF.head()

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
53360,A3OI7ZGH6WZJ5G,0006641040,"Mary Jane Rogers ""Maedchen""",1293840000,5,author wrote wild thing carol king wrote great...,2
26134,A2BEAAU6N7K8GD,B000052Y74,Silne30,1350345600,4,describ minti help milddri mouth provid refres...,2
19211,A1YVJENMZL29O7,B00005344V,R. Meuli,1282348800,5,product awesom help anyon asthma breath diffic...,3
473,A10K7E9F0THV3,B0000535LF,Carol E. White,1315180800,5,naturad total soy meal replac powder advertis ...,4
41746,A33BL538BKBA0S,B00005C2M3,girlabouttownl33t,1301443200,5,buy much cost effect way sinc lot make great u...,2
